# Multi-step LSTM Model (Vector Output Model)

Like other types of neural network models, the LSTM can output a vector directly that can
be interpreted as a multi-step forecast. This approach was seen in the previous section were
one time step of each output time series was forecasted as a vector. As with the LSTMs for
univariate data in a prior section, the prepared samples must first be reshaped. The LSTM
expects data to have a three-dimensional structure of [samples, timesteps, features], and
in this case, we only have one feature so the reshape is straightforward.

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


#### split a univariate sequence into samples

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

## 1. Data Preparation

#### define input sequence

In [3]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

#### choose a number of time steps

In [4]:
n_steps_in, n_steps_out = 3, 2

#### convert into input/output

In [5]:
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [6]:
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


#### reshape from [samples, timesteps] into [samples, timesteps, features]

In [7]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

## 2. Stacked LSTM

#### define model

In [8]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

#### fit model

In [9]:
model.fit(X, y, epochs=50, verbose=0)

#### demonstrate prediction

In [10]:
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[112.66037 134.1638 ]]
